In [1]:
import numpy as np
from sklearn import preprocessing

In [3]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')
unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:,-1]

In [4]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_all = np.delete(targets_all, indices_to_remove, axis = 0)

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [8]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_all[shuffled_indices]

In [10]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count: train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count: train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1801.0 3579 0.5032131880413523
215.0 447 0.4809843400447427
221.0 448 0.49330357142857145


In [11]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [12]:
import tensorflow as tf

In [14]:
npz = np.load('Audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

In [20]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
        tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
        tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
        tf.keras.layers.Dense(output_size, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

BATCH_SIZE = 100
EPOCHS = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 2)

model.fit(train_inputs, 
          train_targets, 
          batch_size = BATCH_SIZE, 
          epochs = EPOCHS, 
          callbacks = [early_stopping],
          validation_data = (validation_inputs, validation_targets),
          verbose = 2)

Epoch 1/100
36/36 - 0s - loss: 0.5425 - accuracy: 0.7594 - val_loss: 0.4290 - val_accuracy: 0.8479
Epoch 2/100
36/36 - 0s - loss: 0.3606 - accuracy: 0.8776 - val_loss: 0.3705 - val_accuracy: 0.8546
Epoch 3/100
36/36 - 0s - loss: 0.3156 - accuracy: 0.8885 - val_loss: 0.3446 - val_accuracy: 0.8591
Epoch 4/100
36/36 - 0s - loss: 0.2963 - accuracy: 0.8896 - val_loss: 0.3275 - val_accuracy: 0.8680
Epoch 5/100
36/36 - 0s - loss: 0.2831 - accuracy: 0.8972 - val_loss: 0.3149 - val_accuracy: 0.8725
Epoch 6/100
36/36 - 0s - loss: 0.2727 - accuracy: 0.8991 - val_loss: 0.3012 - val_accuracy: 0.8814
Epoch 7/100
36/36 - 0s - loss: 0.2657 - accuracy: 0.8997 - val_loss: 0.2887 - val_accuracy: 0.8881
Epoch 8/100
36/36 - 0s - loss: 0.2587 - accuracy: 0.9030 - val_loss: 0.2849 - val_accuracy: 0.8859
Epoch 9/100
36/36 - 0s - loss: 0.2558 - accuracy: 0.9053 - val_loss: 0.2820 - val_accuracy: 0.8904
Epoch 10/100
36/36 - 0s - loss: 0.2506 - accuracy: 0.9058 - val_loss: 0.2780 - val_accuracy: 0.8859
Epoch 11/

In [21]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.2482 - accuracy: 0.9040


In [22]:
print('\n Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))


 Test loss: 0.25. Test accuracy: 90.40%
